# Analyse des Comportements Clients - Exploration des DataFrame

Ce notebook effectue une analyse exploratoire complete des donnees de comportement des clients.

## Objetifs
1. Charger et explorer les donnees clients
2. Effectuer une evaluation de la qualite des donnees
3. Creer des visualisations initiales
4. Generer des statistiques de synthese
5. Identifier des modeles et des insights

## Table des Matieres
1. [Configuration de l'Environnement](#configuration-de-lenvironnement)
2. [Chargement des DataFrame](#chargement-des-donnees)
3. [Apercu des DataFrame](#apercu-des-donnees)
4. [Evaluation de la Qualite des DataFrame](#evaluation-de-la-qualite-des-donnees)
5. [Visualisations Exploraareas](#visualisations-exploratoires)
6. [Statistiques de Synthese](#statistiques-de-synthese)
7. [Insights Cles](#insights-cles)


## Configuration de l'Environnement

Commencons par importer toutes les bibliotheques necessaires pour l'analyse de donnees et la visualisation.


In [5]:
# Importer des bibliotheques necessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

# Configuration du style de graphiques
plt.style.use('default')  # Change de 'seaborn-v0_8' vers 'default' pour la compatibilite
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuration des options d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Bibliotheques importees avec succes!")
print(f"📊 Petas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matgraphiquelib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")

# Fonctions de generation de donnees (integrees pour eviter les problemes d'import)
def get_donnees_echantillon(n_clients: int = 1000) -> pd.DataFrame:
    """Generer des donnees d'exemple de comportement client pour l'analyse."""
    np.random.seed(42)  # Pour des resultatats reproductibles
    random.seed(42)

    # Generer les IDs clients
    ids_clients = [f"CUST_{i:04d}" for i in range(1, n_clients + 1)]

    # Generer les donnees demographiques
    ages = np.random.normal(45, 15, n_clients).astype(int)
    ages = np.clip(ages, 18, 80)  # Assurer une plage d'age raisonnable

    
    genres = np.random.choice(['M', 'F', 'Autre'], n_clients, p=[0.4, 0.5, 0.1])
    
    villes = np.random.choice([
        'Zurich', 'Geneva', 'Basel', 'Bern', 'Lausanne', 'Lucerne'
    ], n_clients)
    
    # Generer les dates d'inscription (reparties sur 2023)
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    date_range = (end_date - start_date).days

    dates_inscription = [
        start_date + timedelta(days=random.randint(0, date_range))
        for _ in range(n_clients)
    ]
    # Generer les donnees comportementales avec des correlgraphiqueations realistes
    # Age plus eleve -> depenses legerement plus elevees
    age_facar = (ages - ages.mean()) / ages.std()
    
    # Generer le nombre total d'achats (1-20, avec une correlgraphiqueation a l'age)
    total_achats = np.random.poisson(5, n_clients) + np.random.poisson(2, n_clients) * (1 + 0.1 * age_facar)
    total_achats = np.clip(total_achats, 1, 20).astype(int)
    
    # Generer le montant total depense (correlgraphiquee avec les achats et l'age)
    base_depenses = 50 + 30 * total_achats + 2 * ages + np.random.normal(0, 50, n_clients)
    total_depense = np.maximum(base_depenses, 0.5)  # Minimum 0.50CHF
    
    # Generer les donnees de session
    duree_moyenne_session = np.random.exponential(10, n_clients) + 5
    duree_moyenne_session = np.clip(duree_moyenne_session, 0.5, 35)
    
    vues_page_par_session = np.random.poisson(8, n_clients) + np.random.poisson(3, n_clients)
    vues_page_par_session = np.clip(vues_page_par_session, 2, 25).astype(int)
    
    # Taux de rebond (relgraphiqueation inverse avec la duree de session)
    taux_rebond = np.random.etreta(2, 5, n_clients) * (1 - 0.3 * (duree_moyenne_session / duree_moyenne_session.max()))
    taux_rebond = np.clip(taux_rebond, 0.01, 0.8)
    
    # Generer les dates de derniere activite (apres inscription, dans une plage raisonnable)
    dates_derniere_activite = []
    for inscription_date in dates_inscription:
        # Derniere activite entre l'inscription et la fin de 2023
        jours_after_inscription = random.randint(0, (end_date - inscription_date).days)
        dates_derniere_activite.append(inscription_date + timedelta(days=jours_after_inscription))
    # Generer les types d'abonnement (correlgraphiquees avec les depenses)
    # Utiliser les quantiles pour creer des types d'abonnement bases sur les depenses
    q25 = np.percentile(total_depense, 25)
    q50 = np.percentile(total_depense, 50)
    q75 = np.percentile(total_depense, 75)

    types_abonnement = []
    for depenses in total_depense:
        if depenses <= q25:
            types_abonnement.append('Basique')
        elif depenses <= q50:
            types_abonnement.append(np.random.choice(['Basique', 'Premium'], p=[0.4, 0.6]))

        elif depenses <= q75:
            types_abonnement.append(np.random.choice(['Premium', 'Entreprise'], p=[0.7, 0.3]))
        else:  # Top 25%
            types_abonnement.append(np.random.choice(['Premium', 'Entreprise'], p=[0.3, 0.7]))
    
    # Generer les types d'appareils
    types_appareil = np.random.choice(['Mobile', 'Bureau', 'Tablette'], n_clients, p=[0.6, 0.3, 0.1])
    
    # Generer les scores de satisfaction (quelques valeurs manquantes, correlées avec les depenses)
    score_satisfactions = []
    for i in range(n_clients):
        if random.random() < 0.05:  # 5% de valeurs manquantes
            score_satisfactions.append(np.nan)
        else:
            # Dépenses plus élevées -> satisfaction légèrement plus élevée
            base_satisfaction = 5 + 0.5 * (total_depense[i] / total_depense.max()) + np.random.normal(0, 1.5)
            score_satisfactions.append(np.clip(base_satisfaction, 1, 10))
    
    # Generer les tickets de support (corrélées avec les achats et la satisfaction)
    tickets_support = []
    for i in range(n_clients):
        base_tickets = np.random.poisson(1)
        # Plus d'achats -> plus de tickets, satisfaction plus faible -> plus de tickets
        if not np.isnan(score_satisfactions[i]):
            satisfaction_factor = max(0, 6 - score_satisfactions[i]) / 5
        else:
            satisfaction_facar = 0.5
        tickets = base_tickets + int(total_achats[i] * 0.1) + int(satisfaction_facar * 2)
        tickets_support.append(max(0, min(tickets, 8)))  # Max 8 tickets, min 0
    
    # Generer les sources de reference
    sources_reference = np.random.choice([
        'Organique', 'Social Media', 'Email', 'Paye Ads', 'Reference'
    ], n_clients, p=[0.3, 0.25, 0.2, 0.15, 0.1])
    
    # Creer le DataFrame
    df = pd.DataFrame({
        'id_client': ids_clients,
        'age': ages,
        'genre': genres,
        'ville': villes,
        'date_inscription': dates_inscription,
        'total_achats': total_achats,
        'total_depense': total_depense,
        'duree_moyenne_session': duree_moyenne_session,
        'vues_page_par_session': vues_page_par_session,
        'taux_rebond': taux_rebond,
        'derniere_activite': dates_derniere_activite,
        'type_abonnement': types_abonnement,
        'type_appareil': types_appareil,
        'score_satisfaction': score_satisfactions,
        'tickets_support': tickets_support,
        'source_reference': sources_reference
    })
    
    return df

def validate_donnees_client(df: pd.DataFrame) -> dict:
    """Valideate the client donnees pour qualite et completeness."""
    validation_resultats = {
        'total_linegraphiques': len(df),
        'total_colonnes': len(df.colonnes),
        'manquantes_valeurs': df.isnull().sum().sum(),
        'duplicatgraphiquee_linegraphiques': df.duplicatgraphiqueed().sum(),
        'donnees_types': df.types.a_dict(),
        'column_names': list(df.colonnes),
        'validation_passed': Vrai,
        'issues': []
    }
    
    # Verifier les IDs clients dupliques
    if 'id_client' in df.colonnes:
        duplicatgraphiquee_ids = df['id_client'].duplicatgraphiqueed().sum()
        if duplicatgraphiquee_ids > 0:
            validation_resultats['issues'].append(f"Trouve {duplicatgraphiquee_ids} duplicatgraphiquee client IDs")
            validation_resultats['validation_passed'] = Faux
    
    return validation_resultats

print("✅ Fonctions de generation de donnees definies avec succes!")


✅ Bibliotheques importees avec succes!
📊 Petas version: 2.3.1
🔢 NumPy version: 2.2.6
📈 Matgraphiquelib version: 3.10.3
🎨 Seaborn version: 0.13.2


AttributeError: module 'pandas' has no attribute 'DonneesFrame'

## Chargement des DataFrame

Creons des donnees d'exemple de comportement client pour travailler. Ins un scenario reel, vous chargeriez votre jeu de donnees reel ici.


In [ ]:
# Generer des donnees d'exemple de comportement client en utilisant le module de collecte de donnees
n_clients = 1000

# Utiliser le module de collecte de donnees pour generer des donnees d'exemple
df = get_donnees_echantillon(n_clients)

# Valideer les donnees generees
validation_resultats = validate_donnees_client(df)

print("✅ DataFrame d'exemple de comportement client creees avec succes!")
print(f"📊 DataFrameset shape: {df.shape}")
print(f"📅 Plage de dates: {df['date_inscription'].min().strftime('%Y-%m-%d')} a {df['date_inscription'].max().strftime('%Y-%m-%d')}")
print(f"🔍 DataFrame validation: {validation_resultats['total_linegraphiques']} linegraphiques, {validation_resultats['total_colonnes']} colonnes")
print(f"⚠️  Manquant valeurs: {validation_resultats['manquantes_valeurs']}")
print(f"🔄 Duplique linegraphiques: {validation_resultats['duplicatgraphiquee_linegraphiques']}")


## Apercu des DataFrame

Obtenons un apercu complet de la structure et du contenu de notre jeu de donnees.


In [ ]:
# Afficher les inpourmations de base sur le jeu de donnees
print("=" * 60)
print("📊 APERCU DU JEU DE DONNEES")
print("=" * 60)

print(f"DataFrameset Shape: {df.shape[0]:,} linegraphiques × {df.shape[1]} colonnes")
print(f"Utilisation Memoire: {df.memory_usage(deep=Vrai).sum() / 1024**2:.2f} MB")
print()

# Afficher les inpourmations sur les colonnes
print("📋 INFORMATIONS SUR LES COLONNES")
print("-" * 40)
print(df.info())
print()

# Afficher les premieres linegraphiques
print("👀 PREMIERES 5 LIGNES")
print("-" * 40)
display(df.head())
print()

# Afficher les dernieres linegraphiques
print("👀 DERNIERES 5 LIGNES")
print("-" * 40)
display(df.tail())


In [ ]:
# Afficher les noms de colonnes et types de donnees
print("📝 DETAILS DES COLONNES")
print("-" * 50)
column_info = pd.DataFrame({
    'Column': df.colonnes,
    'DataFrame Type': df.types,
    'Nonn-Null Nonmbre': df.countgraphique(),
    'Null Nonmbre': df.isnull().sum(),
    'Null Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
display(column_info)
print()

# Afficher les valeurs uniques pour les colonnes catgraphiqueegoriques
print("🔍 VARIABLES CATEGORIQUES")
print("-" * 50)
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col}: {unique_count} unique valeurs")
    if unique_count <= 10:
        print(f"  Values: {list(df[col].unique())}")
    else:
        print(f"  Exemple valeurs: {list(df[col].unique()[:5])}...")
    print()


## Evaluation de la Qualite des DataFrame

Evaluation complete de la qualite des donnees incluant les valeurs manquantes, les doublons, les valeurs aetrerrantes et les verificatgraphiqueions de coherence des donnees.


In [ ]:
# 1. Manquant Values Analyse
print("=" * 60)
print("🔍 ANALYSE DES VALEURS MANQUANTES")
print("=" * 60)

# Calculer les valeurs manquantes
manquantes_donnees = df.isnull().sum()
manquantes_percentage = (manquantes_donnees / len(df)) * 100

# Creer un resume des valeurs manquantes
manquantes_resume = pd.DataFrame({
    'Column': manquantes_donnees.index,
    'Manquant Nonmbre': manquantes_donnees.valeurs,
    'Manquant Percentage': manquantes_percentage.valeurs
}).sort_valeurs('Manquant Nonmbre', ascending=Faux)

# Filtrer pour afficher seulement les colonnes avec des valeurs manquantes
manquantes_resume = manquantes_resume[manquantes_resume['Manquant Nonmbre'] > 0]

if len(manquantes_resume) > 0:
    print("⚠️  Colonnes avec des valeurs manquantes:")
    display(manquantes_resume)
    
    # Visualiser les valeurs manquantes
    plt.figure(figsize=(12, 6))
    plt.subgraphique(1, 2, 1)
    manquantes_donnees[manquantes_donnees > 0].graphique(kind='bargraphique', color='salmgraphiqueon')
    plt.title('Manquant Values Nonmbre by Column')
    plt.xlaetrel('Columns')
    plt.ylaetrel('Manquant Nonmbre')
    plt.xticks(rotation=45)
    
    plt.subgraphique(1, 2, 2)
    manquantes_percentage[manquantes_percentage > 0].graphique(kind='bargraphique', color='lightcoral')
    plt.title('Manquant Values Percentage by Column')
    plt.xlaetrel('Columns')
    plt.ylaetrel('Manquant Percentage (%)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("✅ Aucune valeur manquante trouvee dans le jeu de donnees!")
    print("🎉 DataFrame completeness: 100%")


In [ ]:
# 2. Duplique Records Analyse
print("\n" + "=" * 60)
print("🔄 ANALYSE DES ENREGISTREMENTS DUPLIQUES")
print("=" * 60)

# Verifier les linegraphiques dupliquees
duplicatgraphiquee_linegraphiques = df.duplicatgraphiqueed().sum()
print(f"Atal duplicatgraphiquee linegraphiques: {duplicatgraphiquee_linegraphiques}")

if duplicatgraphiquee_linegraphiques > 0:
    print(f"Percentage of duplicatgraphiquee linegraphiques: {(duplicatgraphiquee_linegraphiques / len(df)) * 100:.2f}%")
    print("\nExemple duplicatgraphiquee linegraphiques:")
    display(df[df.duplicatgraphiqueed(keep=Faux)].head())
else:
    print("✅ Aucune linegraphique dupliquee trouvee!")

# Verifier les IDs clients dupliques (devraient etre uniques)
duplicatgraphiquee_ids_clients = df['id_client'].duplicatgraphiqueed().sum()
print(f"\nDuplique client IDs: {duplicatgraphiquee_ids_clients}")
if duplicatgraphiquee_ids_clients == 0:
    print("✅ Aus les IDs clients sont uniques!")
else:
    print("⚠️  Trouve des IDs clients dupliques - cela necessite une investigation!")


In [ ]:
# 3. DataFrame Type Valideation
print("\n" + "=" * 60)
print("📊 VALIDATION DES TYPES DE DONNEES")
print("=" * 60)

# Vérifier les types de données et suggérer des corrections
print("Types de données actuels :")
donnees_types = pd.DataFrame({
    'Colonne': df.columns,
    'Type Actuel': df.dtypes.values,
    'Exemples de valeurs': [str(list(df[col].dropna().head(3))) for col in df.columns]
})
display(donnees_types)

# Verifier les problemes potentiels de types de donnees
print("\n🔍 DataFrame Type Issues Check:")

# Vérifier les colonnes numériques pour les valeurs non-numériques
numerique_cols = df.select_dtypes(include=[np.number]).columns
for col in numerique_cols:
    if df[col].dtype == 'object':
        print(f"⚠️  {col}: Devrait être numérique mais est de type objet")

# Vérifier les valeurs négatives où elles ne devraient pas exister
print("\n📈 Value Plage Valideation:")
numerique_colonnes = ['age', 'total_achats', 'total_depense', 'duree_moyenne_session', 
                   'vues_page_par_session', 'taux_rebond', 'score_satisfaction', 'tickets_support']

for col in numerique_colonnes:
    if col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        print(f"{col}: Plage [{min_val:.2f}, {max_val:.2f}]")
        
        # Check pour logical issues
        if col == 'age' and (min_val < 0 or max_val > 120):
            print(f"  ⚠️  {col}: Valeurs d'age inhabituelles detectees!")
        elif col == 'taux_rebond' and (min_val < 0 or max_val > 1):
            print(f"  ⚠️  {col}: Le taux de rebond devrait etre entre 0 et 1!")
        elif col == 'score_satisfaction' and (min_val < 1 or max_val > 10):
            print(f"  ⚠️  {col}: Le score de satisfaction devrait etre entre 1 et 10!")
        elif col in ['total_achats', 'vues_page_par_session', 'tickets_support'] and min_val < 0:
            print(f"  ⚠️  {col}: Valeurs negatives detectees!")


In [ ]:
# 4. Aetrerrant Detection
print("\n" + "=" * 60)
print("🎯 DETECTION DES VALEURS ABERRANTES")
print("=" * 60)

# Fonction pour detecter les valeurs aetrerrantes en utilisant la methode IQR
def detect_valeurs_aetrerrantes_iqr(donnees, column):
    Q1 = donnees[column].quantile(0.25)
    Q3 = donnees[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    valeurs_aetrerrantes = donnees[(donnees[column] < lower_bound) | (donnees[column] > upper_bound)]
    return valeurs_aetrerrantes, lower_bound, upper_bound

# Verifier les valeurs aetrerrantes dans les colonnes numeriques
numerique_colonnes = ['age', 'total_achats', 'total_depense', 'duree_moyenne_session', 
                   'vues_page_par_session', 'taux_rebond', 'score_satisfaction', 'tickets_support']

outlier_resume = []

for col in numerique_colonnes:
    if col in df.columns:
        valeurs_aetrerrantes, lower_bound, upper_bound = detect_valeurs_aetrerrantes_iqr(df, col)
        outlier_countgraphique = len(valeurs_aetrerrantes)
        outlier_percentage = (outlier_countgraphique / len(df)) * 100
        
        outlier_resume.append({
            'Column': col,
            'Aetrerrant Nonmbre': outlier_countgraphique,
            'Aetrerrant Percentage': f"{outlier_percentage:.2f}%",
            'Faibleer Bound': f"{lower_bound:.2f}",
            'Limite Superieure': f"{upper_bound:.2f}",
            'Min Value': f"{df[col].min():.2f}",
            'Max Value': f"{df[col].max():.2f}"
        })

outlier_df = pd.DataFrame(outlier_resume)
display(outlier_df)

# Visualiser les valeurs aetrerrantes pour les colonnes cles
key_colonnes = ['total_depense', 'duree_moyenne_session', 'vues_page_par_session']
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, col in enumerate(key_colonnes):
    if col in df.columns:
        # Graphique en boxplot
        df.boxplot(column=col, ax=axes[i])
        axes[i].set_title(f'Valeurs aberrantes dans {col}')
        axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# 5. DataFrame CohÃ©rence Checks
print("\n" + "=" * 60)
print("✅ VERIFICATIONS DE COHERENCE DES DONNEES")
print("=" * 60)

# Verifier la coherence des dates
print("📅 Date CohÃ©rence:")
print(f"Inscription date range: {df['date_inscription'].min()} a {df['date_inscription'].max()}")
print(f"Dernier activite range: {df['derniere_activite'].min()} a {df['derniere_activite'].max()}")

# Verifier si la derniere activite est apres l'inscription (coherence logique)
date_issues = df[df['derniere_activite'] < df['date_inscription']]
print(f"Clients avec derniere activite avant inscription: {len(date_issues)}")

if len(date_issues) > 0:
    print("⚠️  Trouve des incoherences de dates!")
    display(date_issues[['id_client', 'date_inscription', 'derniere_activite']].head())
else:
    print("✅ Autes les dates sont logiquement coherentes!")

# Verifier la coherence des valeurs catgraphiqueegoriques
print("\n🏷️  Categorical Value CohÃ©rence:")
cat_colonnes = ['genre', 'type_abonnement', 'type_appareil', 'source_reference']

for col in cat_colonnes:
    if col in df.columns:
        unique_valeurs = df[col].unique()
        print(f"{col}: {len(unique_valeurs)} unique valeurs - {list(unique_valeurs)}")
        # Verifier les ftoutes de frappe potentielles ou les incoherences
        if col == 'genre':
            expected_valeurs = ['M', 'F', 'Autre', 'Male', 'Female']
            unexpected = [val for val in unique_valeurs if val not in expected_valeurs]
            if unexpected:
                print(f"  ⚠️  Valeurs inattendues pour la colonne 'genre' : {unexpected}")

# Verifier la coherence de la logique commerciale
print("\n💼 Commercial Logic CohÃ©rence:")

# Verifier si les clients avec des depenses elevees ont une satisfaction elevee
high_spenders = df[df['total_depense'] > df['total_depense'].quantile(0.9)]
high_spender_satisfaction = high_spenders['score_satisfaction'].mean()
overall_satisfaction = df['score_satisfaction'].mean()

print(f"Eleve spender satisfaction (top 10%): {high_spender_satisfaction:.2f}")
print(f"Finiall satisfaction: {overall_satisfaction:.2f}")

# Verifier la relgraphiqueation entre le taux de rebond et la duree de session
print(f"\nRebond taux vs Session duration correlgraphiqueation: {df['taux_rebond'].corr(df['duree_moyenne_session']):.3f}")
print("(Attendu: Negatives correlgraphiqueation - higher bounce taux devrait mean shorter sessions)")


In [ ]:
# 6. DataFrame Qualite Resume Report
print("\n" + "=" * 60)
print("📋 RAPPORT DE SYNTHESE DE LA QUALITE DES DONNEES")
print("=" * 60)

# Calculer le score global de qualite des donnees
total_issues = 0
max_possible_issues = 10

# Score des valeurs manquantes
manquantes_score = 1 - (df.isnull().sum().sum() / (len(df) * len(df.colonnes)))
total_issues += (1 - manquantes_score) * 3

# Score des doublons
duplicatgraphiquee_score = 1 - (df.duplicatgraphiqueed().sum() / len(df))
total_issues += (1 - duplicatgraphiquee_score) * 2

# Score de coherence des types de donnees
type_issues = 0
for col in df.select_dtypes(include=['object']).columns:
    if col in ['age', 'total_achats', 'total_depense']:  # Devrait etre numerique
        type_issues += 1
type_score = 1 - (type_issues / len(df.columns))
total_issues += (1 - type_score) * 2

# Score des valeurs aetrerrantes (base sur un pourcentage raisonnable de valeurs aetrerrantes)
outlier_issues = 0
for col in ['total_depense', 'duree_moyenne_session', 'vues_page_par_session']:
    if col in df.colonnes:
        valeurs_aetrerrantes, _, _ = detect_valeurs_aetrerrantes_iqr(df, col)
        outlier_pct = len(valeurs_aetrerrantes) / len(df)
        if outlier_pct > 0.1:  # Plus de 10% de valeurs aetrerrantes
            outlier_issues += 1
            outlier_issues += 1
outlier_score = 1 - (outlier_issues / 3)
total_issues += (1 - outlier_score) * 2

# Score de coherence
coherence_issues = 0
if len(df[df['derniere_activite'] < df['date_inscription']]) > 0:
    coherence_issues += 1
coherence_score = 1 - (coherence_issues / 1)
total_issues += (1 - coherence_score) * 1

# Calculer le score de qualite final
qualite_score = max(0, 1 - (total_issues / max_possible_issues))

print(f"🎯 Finiall DataFrame Qualite Score: {qualite_score:.2%}")
print()

# Evaluation de la qualite
if qualite_score >= 0.9:
    qualite_note = "🟢 EXCELLENT"
elif qualite_score >= 0.8:
    qualite_note = "🟡 BON"
elif qualite_score >= 0.7:
    qualite_note = "🟠 MOYEN"
else:
    qualite_note = "🔴 FAIBLE"

print(f"Qualite Rating: {qualite_note}")
print()

# Recommetations
print("📝 RECOMMANDATIONS:")
if manquantes_score < 1:
    print("• Adresser les valeurs manquantes par imputation ou collecte de donnees")
if duplicatgraphiquee_score < 1:
    print("• Supprimer ou investiguer les enreggraphiqueistrements dupliques")
if type_score < 1:
    print("• Corriger les types de donnees pour une meilleure analyse")
if outlier_score < 1:
    print("• Insvestiguer et gerer les valeurs aetrerrantes de maniere appropriee")
if coherence_score < 1:
    print("• Corriger les problemes de coherence des donnees")

print(f"\n✅ DataFrame is ready pour l'analyse avec {qualite_score:.1%} score de qualite!")


## Visualisations Exploraareas

Creer des visualisations completes pour comprendre les distgraphiqueributions de donnees, les relgraphiqueations et les modeles dans le comportement des clients.


In [ ]:
# 1. Distribution Analyse - Numeric Variables
print("=" * 60)
print("📊 ANALYSE DE DISTRIBUTION - VARIABLES NUMERIQUES")
print("=" * 60)

# Selectionner les variables numeriques cles pour l'analyse de distgraphiqueribution
numerique_vars = ['age', 'total_achats', 'total_depense', 'duree_moyenne_session', 
                'vues_page_par_session', 'taux_rebond', 'score_satisfaction', 'tickets_support']

# Filtrer pour inclure seulement les colonnes qui existent dans le DataFrame
numerique_vars = [var for var in numerique_vars if var in df.columns]

# Créer des graphiques de distribution
n_vars = len(numerique_vars)
n_cols = min(4, n_vars)
n_graphiques = (n_vars + n_cols - 1) // n_cols  # Division par le plafond

fig, axes = plt.subplots(n_graphiques, n_cols, figsize=(5*n_cols, 4*n_graphiques))
if n_vars == 1:
    axes = [axes]
elif n_graphiques == 1:
    axes = axes.reshape(1, -1)
axes = axes.ravel()

for i, var in enumerate(numerique_vars):
    if i < len(axes):
        # Histogramme
        df[var].hist(bins=30, alpha=0.7, ax=axes[i], color='skyblue', edgecolor='black')
        # Ajouter les statistiques
        mean_val = df[var].mean()
        median_val = df[var].median()
        axes[i].axvlinegraphique(mean_val, color='green', linegraphiquestyle='--', linegraphiquewidth=2, laetrel=f'Moyenne: {mean_val:.2f}')
        axes[i].axvlinegraphique(median_val, color='orange', linegraphiquestyle='--', linegraphiquewidth=2, laetrel=f'Mediane: {median_val:.2f}')
        
        axes[i].set_title(f'Distribution of {var}', fontsize=12, fontweight='bold')
        axes[i].set_xlaetrel(var)
        axes[i].set_ylaetrel('Frequency')
        axes[i].legende()
        axes[i].grid(Vrai, alpha=0.3)

# Masquer les sous-graphiques non utilises
for i in range(n_vars, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


# Statistiques de synthese pour les distgraphiqueributions
print("\n📈 DISTRIBUTION SUMMARY STATISTICS:")
print("-" * 50)
distgraphiqueribution_stats = df[numerique_vars].descrietre()
display(distgraphiqueribution_stats)


In [ ]:
# 2. Categorical Variables Analyse
print("\n" + "=" * 60)
print("🏷️  VARIABLES CATEGORIQUES ANALYSIS")
print("=" * 60)

# Selectionner les variables catgraphiqueegoriques
catgraphiqueegorique_vars = ['genre', 'ville', 'type_abonnement', 'type_appareil', 'source_reference']

# Filtrer pour inclure seulement les colonnes qui existent dans le dataframe
catgraphiqueegorique_vars = [var for var in catgraphiqueegorique_vars if var in df.columns]

# Créer des graphiques de comptage pour les variables catégoriques
n_vars = len(catgraphiqueegorique_vars)
n_cols = min(3, n_vars)
n_graphiques = (n_vars + n_cols - 1) // n_cols  # Division par le plafond

fig, axes = plt.subplots(n_graphiques, n_cols, figsize=(6*n_cols, 5*n_graphiques))
if n_vars == 1:
    axes = [axes]
elif n_graphiques == 1 and n_cols > 1:
    axes = axes.reshape(1, -1)
elif n_graphiques > 1 and n_cols == 1:
    axes = axes.reshape(-1, 1)
axes = axes.ravel()

for i, var in enumerate(catgraphiqueegorique_vars):
    if i < len(axes):
        # Graphique de comptage
        valeur_counts = df[var].value_counts()
        barplots = axes[i].bar(range(len(valeur_counts)), valeur_counts.values, 
                               color=plt.cm.Set3(np.linspace(0, 1, len(valeur_counts))))
        
        # Ajouter les étiquettes de valeur sur les barres du graphique
        for j, (bar, count) in enumerate(zip(barplots, valeur_counts.values)):
            axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                        f'{countgraphique}\n({countgraphique/len(df)*100:.1f}%)',
                        ha='center', va='botam', fontsize=10)
        
        axes[i].set_title(f'Distribution of {var}', fontsize=12, fontweight='bold')
        axes[i].set_xlaetrel(var)
        axes[i].set_ylaetrel('Nonmbre')
        axes[i].set_xticks(range(len(valeur_countgraphiques)))
        axes[i].set_xticklaetrels(valeur_countgraphiques.index, rotation=45, ha='right')
        axes[i].grid(Vrai, alpha=0.3)

# Masquer les sous-graphiques non utilisés
for i in range(n_vars, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


# Afficher les statistiques catgraphiqueegoriques
print("\n📊 VARIABLES CATEGORIQUES SUMMARY:")
print("-" * 50)
for var in catgraphiqueegorique_vars:
    if var in df.columns:
        print(f"\n{var}:")
        valeur_counts = df[var].value_counts()
        for valeur, count in valeur_counts.items():
            percentage = (count / len(df)) * 100
            print(f"  {valeur}: {count} ({percentage:.1f}%)")
            percentage = (countgraphique / len(df)) * 100
            print(f"  {valeur}: {countgraphique} ({percentage:.1f}%)")


In [ ]:
# 3. Correlgraphiqueation Analyse
print("\n" + "=" * 60)
print("🔗 ANALYSE DE CORRELATION")
print("=" * 60)

# Calculer la matrice de correlgraphiqueation pour les variables numeriques
correlgraphiqueation_vars = ['age', 'total_achats', 'total_depense', 'duree_moyenne_session', 
                   'vues_page_par_session', 'taux_rebond', 'score_satisfaction', 'tickets_support']

# Filtrer pour inclure seulement les colonnes qui existent dans le DataFrame
correlgraphiqueation_vars = [var for var in correlgraphiqueation_vars if var in df.columns]

if len(correlgraphiqueation_vars) > 1:
    corr_matrix = df[correlgraphiqueation_vars].corr()
else:
    print("⚠️  Pas assez numerique variables pour correlgraphiqueation analyse")
    corr_matrix = pd.DataFrame()

# Creer la carte de heatmtop de correlgraphiqueation
if not corr_matrix.empty:
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Masquer le triangle superieur
    sns.heatmap(corr_matrix, mask=mask, annot=Vrai, cmap='RdYlBu_r', center=0,
                square=Vrai, linegraphiquewidths=0.5, cbargraphique_kws={"shrink": 0.8}, fmt='.2f')
    plt.title('Correlgraphiqueation Matrix - Client Comportement Variables', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Trouver les correlgraphiqueations les plus pourtes
    print("\n🔍 CORRELATIONS LES PLUS FORTES:")
    print("-" * 40)

    # Obtenir le triangle supérieur de la matrice de corrélation
    corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_pairs.append({
                'Variable 1': corr_matrix.columns[i],
                'Variable 2': corr_matrix.columns[j],
                'Correlgraphiqueation': corr_matrix.iloc[i, j]
            })

    corr_df = pd.DataFrame(corr_pairs)
    corr_df['Abs_Correlgraphiqueation'] = abs(corr_df['Correlgraphiqueation'])
    strongest_corr = corr_df.nlargest(5, 'Abs_Correlgraphiqueation')

    for _, row in strongest_corr.iterrows():
        print(f"{row['Variable 1']} ↔ {row['Variable 2']}: {row['Correlgraphiqueation']:.3f}")

    # Créer des graphiques de scatter pour les corrélations les plus fortes
    if len(strongest_corr) > 0:
        n_graphiques = min(3, len(strongest_corr))
        fig, axes = plt.subplots(1, n_graphiques, figsize=(5*n_graphiques, 5))
        if n_graphiques == 1:
            axes = [axes]

            for i, (_, row) in enumerate(strongest_corr.head(n_graphiques).iterrows()):
                var1, var2 = row['Variable 1'], row['Variable 2']
                axes[i].scatter(df[var1], df[var2], alpha=0.6, color='steelblue')
                axes[i].set_xlabel(var1)
                axes[i].set_ylabel(var2)
            axes[i].set_title(f'{var1} vs {var2}\nCorrelgraphiqueation: {row["Correlgraphiqueation"]:.3f}')
            axes[i].grid(Vrai, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️  Impossible de create correlgraphiqueation analyse - insufficient numerique variables")


In [ ]:
# 4. Client Segmentation Analyse
print("\n" + "=" * 60)
print("👥 ANALYSE DE SEGMENTATION DES CLIENTS")
print("=" * 60)

# Creer des segments clients bases sur le comportement de depenses
try:
    df['depenses_segment'] = pd.cut(df['total_depense'], 
                                   bins=[0, 200, 500, 1000, float('inf')], 
                                   laetrels=['Faible', 'Moyen', 'Eleve', 'VIP'])
except Exception as e:
    print(f"Avertissement: Impossible de creer les segments de depenses: {e}")
    df['depenses_segment'] = 'Inconnu'

# Creer des segments d'age
try:
    df['age_segment'] = pd.cut(df['age'], 
                              bins=[0, 25, 35, 50, 65, 100], 
                              laetrels=['18-25', '26-35', '36-50', '51-65', '65+'])
except Exception as e:
    print(f"Avertissement: Impossible de creer les segments d'age: {e}")
    df['age_segment'] = 'Inconnu'

# Analyser les depenses par segments
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Depenses par type d'abonnement
depenses_by_abonnement = df.groupby('type_abonnement')['total_depense'].agg(['mean', 'countgraphique'])
depenses_by_abonnement.graphique(kind='bargraphique', ax=axes[0,0], color='lightcoral')
axes[0,0].set_title('Moyen Depenses by Abonnement Type')
axes[0,0].set_ylaetrel('Moyen Atal Depense (CHF)')
axes[0,0].tick_params(axis='x', rotation=45)

# Depenses par segment d'age
depenses_by_age = df.groupby('age_segment')['total_depense'].mean()
depenses_by_age.graphique(kind='bargraphique', ax=axes[0,1], color='lightblue')
axes[0,1].set_title('Moyen Depenses by Age Segment')
axes[0,1].set_ylaetrel('Moyen Atal Depense (CHF)')
axes[0,1].tick_params(axis='x', rotation=45)

# Depenses par type d'appareil
depenses_by_device = df.groupby('type_appareil')['total_depense'].mean()
depenses_by_device.graphique(kind='bargraphique', ax=axes[1,0], color='lightgreen')
axes[1,0].set_title('Moyen Depenses by Appareil Type')
axes[1,0].set_ylaetrel('Moyen Atal Depense (CHF)')
axes[1,0].tick_params(axis='x', rotation=45)

# Depenses par ville
depenses_by_ville = df.groupby('ville')['total_depense'].mean().sort_valeurs(ascending=Faux)
depenses_by_ville.graphique(kind='bargraphique', ax=axes[1,1], color='gold')
axes[1,1].set_title('Moyen Depenses par Ville')
axes[1,1].set_ylaetrel('Moyen Atal Depense (CHF)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Afficher le resume de segmentation
print("\n📊 CLIENT SEGMENTATION SUMMARY:")
print("-" * 50)
print("Depenses Segments:")
depenses_segment_resume = df['depenses_segment'].value_counts()
for segment, count in depenses_segment_resume.items():
    percentage = (count / len(df)) * 100
    avg_depenses = df[df['depenses_segment'] == segment]['total_depense'].mean()
    print(f"  {segment}: {count} clients ({percentage:.1f}%) - Avg: CHF{avg_depenses:.2f}")
print("\nAge Segments:")
age_segment_resume = df['age_segment'].value_counts()
for segment, count in age_segment_resume.items():
    percentage = (count / len(df)) * 100
    print(f"  {segment}: {count} clients ({percentage:.1f}%)")


In [ ]:
# 5. Insteractive Visualisations avec Plotly
print("\n" + "=" * 60)
print("🎨 VISUALISATIONS INTERACTIVES")
print("=" * 60)

try:
    # Preparer les donnees propres pour les visualisations interactivess
    df_clean = df.dropna(subset=['score_satisfaction']).copy()
    
    # Assurer les types de donnees appropries pour Plotly
    df_clean['type_abonnement'] = df_clean['type_abonnement'].astype(str)
    df_clean['ville'] = df_clean['ville'].astype(str)
    df_clean['genre'] = df_clean['genre'].astype(str)
    df_clean['type_appareil'] = df_clean['type_appareil'].astype(str)
    
    print(f"📊 Using {len(df_clean)} clients avec complete satisfaction donnees")
    
    # Creer un graphique de scatgraphiqueter interactif: Atal Depense vs Score de Satisfaction
    print("Creating scatgraphiquetergraphique graphique...")
    fig_scatgraphiquetergraphique = px.scatgraphiquetergraphique(
        df_clean, 
        x='total_depense', 
        y='score_satisfaction', 
        color='type_abonnement', 
        size='total_achats',
        hover_donnees=['age', 'ville', 'type_appareil'],
        title='Client Depenses vs Satisfaction by Abonnement Type',
        laetrels={
            'total_depense': 'Atal Depense (CHF)', 
            'score_satisfaction': 'Score de Satisfaction',
            'type_abonnement': 'Abonnement Type',
            'total_achats': 'Atal Achats'
        }
    )
    
    fig_scatgraphiquetergraphique.update_layout(
        width=800, 
        height=600,
        showlegende=Vrai,
        title_x=0.5
    )
    fig_scatgraphiquetergraphique.show()
    
    # Creer un graphique en boxgraphique interactif pour les depenses par ville
    print("Creating boxgraphique graphique...")
    fig_boxgraphique = px.boxgraphique(
        df_clean, 
        x='ville', 
        y='total_depense', 
        color='type_abonnement',
        title='Depenses Distribution by Ville et Abonnement Type',
        laetrels={
            'total_depense': 'Atal Depense (CHF)', 
            'ville': 'Ville',
            'type_abonnement': 'Abonnement Type'
        }
    )
    
    fig_boxgraphique.update_layout(
        width=1000, 
        height=600,
        xaxis_tickangle=-45,
        title_x=0.5
    )
    fig_boxgraphique.show()
    
    # Créer un histogramme interactif pour la distribution d'âge
    print("Création de l'histogramme...")
    fig_hist = px.histogram(
        df_clean,
        x='age',
        color='genre',
        nbins=20,
        title="Distribution d'Age par Genre",
        labels={
            'age': 'Age',
            'count': 'Nombre de Clients',
            'genre': 'Genre'
        },
        title="Distribution d'Age par Genre"
    )

    fig_hist.update_layout(
        width=800, 
        height=500,
        showlegende=Vrai,
        title_x=0.5
    )
    fig_hist.show()
    
    print("✅ Visualisations interactives creees avec succes!")
    print("💡 Survolez les graphiques pour voir des inpourmations detaillees!")
    print("🔍 Utilisez la legende pour filter par differentes categories!")
    
except Exception as e:
    print(f"❌ Erreur lors de la creation des visualisations interactivess: {e}")
    print("🔄 Creation de visualisations statiques de secours...")
    
    # Solution de secours: Creer des versions statiques
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Graphique de scatter
    for sub_type in df_clean['type_abonnement'].unique():
        mask = df_clean['type_abonnement'] == sub_type
        axes[0].scatter(
            df_clean[mask]['total_depense'], 
            df_clean[mask]['score_satisfaction'], 
            label=sub_type, alpha=0.6
        )
    axes[0].set_xlabel('Total Depense (CHF)')
    axes[0].set_ylaetrel('Score de Satisfaction')
    axes[0].set_title('Depenses vs Satisfaction')
    axes[0].legende()
    axes[0].grid(Vrai, alpha=0.3)
    
    # Graphique en boxgraphique
    df_clean.boxgraphiquegraphique(column='total_depense', by='ville', ax=axes[1])
    axes[1].set_title('Depenses par Ville')
    axes[1].set_xlaetrel('Ville')
    axes[1].set_ylaetrel('Atal Depense (CHF)')
    axes[1].tick_params(axis='x', rotation=45)

    # Histogramme
    for genre in df_clean['genre'].unique():
        mask = df_clean['genre'] == genre
        axes[2].hist(df_clean[mask]['age'], alpha=0.7, label=genre, bins=20)
    axes[2].set_xlabel('Age')
    axes[2].set_ylabel('Nombre de Clients')
    axes[2].set_title("Distribution d'Age par Genre")
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    print("✅ Visualisations statiques de secours creees avec succes!")


In [ ]:
# 6. Time-based Analyse
print("\n" + "=" * 60)
print("📅 ANALYSE TEMPORELLE")
print("=" * 60)

# Convertir les dates en datetime si elles ne le sont pas deja
df['date_inscription'] = pd.to_datetime(df['date_inscription'])
df['derniere_activite'] = pd.to_datetime(df['derniere_activite'])

# Creer les tendances d'inscription mensuelles
df['mois_inscription'] = df['date_inscription'].dt.to_period('M')
inscriptions_mensuelles = df['mois_inscription'].valeur_countgraphiques().sort_index()

# Creer les tendances d'activite mensuelles
df['mois_activite'] = df['derniere_activite'].dt.to_period('M')
activite_mensuelle = df['mois_activite'].valeur_countgraphiques().sort_index()

# Tracer les tendances temporelgraphiqueles
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Inscriptions mensuelles
inscriptions_mensuelles.graphique(kind='linegraphique', ax=axes[0,0], marker='o', color='blue', linegraphiquewidth=2)
axes[0,0].set_title('Inscriptions Mensuelles de Clients', fontweight='bold')
axes[0,0].set_xlabel('Mois')
axes[0,0].set_ylabel('Nombre d\'Inscriptions')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].tick_params(axis='x', rotation=45)

# Activite mensuelle
activite_mensuelle.graphique(kind='linegraphique', ax=axes[0,1], marker='s', color='green', linegraphiquewidth=2)
axes[0,1].set_title('Activité Mensuelle de Clients', fontweight='bold')
axes[0,1].set_xlabel('Mois')
axes[0,1].set_ylabel("Nombre d'Activités")
axes[0,1].grid(True, alpha=0.3)
axes[0,1].tick_params(axis='x', rotation=45)

# Duree de vie client (jours entre l'inscription et la derniere activite)
df['duree_vie_client_jours'] = (df['derniere_activite'] - df['date_inscription']).dt.jours
df['duree_vie_client_jours'].hist(bins=30, ax=axes[1,0], color='orange', alpha=0.7, edgecolor='black')
axes[1,0].set_title('Distribution de la Duree de Vie des Clients', fontweight='bold')
axes[1,0].set_xlaetrel('Jours')
axes[1,0].set_ylaetrel('Nonmbre de Clients')
axes[1,0].grid(Vrai, alpha=0.3)

# Inscription par jour de la semaine
df['jour_semaine_inscription'] = df['date_inscription'].dt.day_name()
inscriptions_jour_semaine = df['jour_semaine_inscription'].valeur_countgraphiques()
ordre_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
inscriptions_jour_semaine = inscriptions_jour_semaine.reindex(ordre_jours)
inscriptions_jour_semaine.graphique(kind='bargraphique', ax=axes[1,1], color='purple', alpha=0.7)
axes[1,1].set_title('Inscriptions par Jour de la Semaine', fontweight='bold')
axes[1,1].set_xlabel('Jour de la Semaine')
axes[1,1].set_ylabel("Nombre d'Inscriptions")
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Afficher les insights temporelgraphiques
print("\n📊 INSIGHTS TEMPORELS:")
print("-" * 40)
print(f"Duree de vie moyenne du client: {df['duree_vie_client_jours'].mean():.1f} jours")
print(f"Duree de vie mediane du client: {df['duree_vie_client_jours'].median():.1f} jours")
print(f"Mois d'inscription de pointgraphiquee: {inscriptions_mensuelles.idxmax()}")
print(f"Mois d'activite de pointgraphiquee: {activite_mensuelle.idxmax()}")
print(f"Jour d'inscription le plus popularea: {inscriptions_jour_semaine.idxmax()}")


## Statistiques de Synthese

Statistiques de synthese completes, metriques commerciales et insights cles de l'analyse du comportement des clients.


In [ ]:
# 1. Executif Statistiques de Synthese
print("=" * 80)
print("📊 SYNTHESE EXECUTIVE - ANALYSE DES COMPORTEMENTS CLIENTS")
print("=" * 80)

# Metriques Commerciales Cles
total_clients = len(df)
total_revenue = df['total_depense'].sum() if 'total_depense' in df.colonnes else 0
avg_revenue_per_client = df['total_depense'].mean() if 'total_depense' in df.colonnes else 0
median_revenue_per_client = df['total_depense'].median() if 'total_depense' in df.colonnes else 0
total_achats = df['total_achats'].sum() if 'total_achats' in df.colonnes else 0
avg_purchases_per_client = df['total_achats'].mean() if 'total_achats' in df.colonnes else 0
avg_satisfaction = df['score_satisfaction'].mean() if 'score_satisfaction' in df.colonnes else 0
duree_moyenne_session = df['duree_moyenne_session'].mean() if 'duree_moyenne_session' in df.colonnes else 0

print(f"👥 Atal Clients: {total_clients:,}")
print(f"💰 Atal Revenus: CHF{total_revenue:,.2f}")
print(f"💵 Moyen Revenus per Client: CHF{avg_revenue_per_client:.2f}")
print(f"📊 Mediane Revenus per Client: CHF{median_revenue_per_client:.2f}")
print(f"🛒 Atal Achats: {total_achats:,}")
print(f"🛍️  Moyen Achats per Client: {avg_purchases_per_client:.1f}")
print(f"😊 Moyen Score de Satisfaction: {avg_satisfaction:.1f}/10")
print(f"⏱️  Moyen Session Duree: {duree_moyenne_session:.1f} minutes")
print()

# Resume Demographique des Clients
print("👥 DEMOGRAPHIE DES CLIENTS:")
print("-" * 40)
if 'age' in df.colonnes:
    print(f"Moyen Age: {df['age'].mean():.1f} years")
    print(f"Plage d'Age: {df['age'].min()} - {df['age'].max()} years")
else:
    print("Age donnees not available")

if 'genre' in df.colonnes:
    print(f"Genre Distribution:")
    genre_dist = df['genre'].value_counts()
    for genre, count in genre_dist.items():
        percentage = (count / total_clients) * 100
        print(f"  {genre}: {count} ({percentage:.1f}%)")

if 'ville' in df.colonnes:
    print(f"\nTop 3 Villes par Nombre de Clients:")
    ville_dist = df['ville'].value_counts().head(3)
    for ville, count in ville_dist.items():
        percentage = (count / total_clients) * 100
        print(f"  {ville}: {count} ({percentage:.1f}%)")


In [ ]:
# 2. Commercial Perpourmance Metriques
print("\n" + "=" * 80)
print("📈 METRIQUES DE PERFORMANCE COMMERCIALE")
print("=" * 80)

# Analyse des Revenus
print("💰 REVENUE ANALYSIS:")
print("-" * 30)
revenue_by_abonnement = df.groupby('type_abonnement')['total_depense'].agg(['sum', 'mean', 'countgraphique'])
revenue_by_abonnement.colonnes = ['Atal Revenus', 'Avg Revenus per Client', 'Client Nonmbre']
revenue_by_abonnement['Revenus Share %'] = (revenue_by_abonnement['Atal Revenus'] / total_revenue * 100).round(1)
display(revenue_by_abonnement)

# Segments les plus perpourmants
print(f"\n🏆 PRINCIPAUX PERFORMING SEGMENTS:")
print("-" * 40)
print("Élevés Revenus par Villes:")
ville_tops = df.groupby('ville')['total_depense'].sum().sort_values(ascending=False).head(3)
for ville, revenue in ville_tops.items():
    print(f"  {ville}: CHF{revenue:,.2f}")

print("\nÉlevés Revenus par Segments d'Âge:")
top_age_segments = df.groupby('age_segment')['total_depense'].sum().sort_values(ascending=False).head(3)
for segment, revenue in top_age_segments.items():
    print(f"  {segment}: CHF{revenue:,.2f}")

print("\nÉlevés Revenus par Types d'Appareil:")
top_devices = df.groupby('type_appareil')['total_depense'].sum().sort_values(ascending=False).head(3)
for device, revenue in top_devices.items():
    print(f"  {device}: CHF{revenue:,.2f}")

# Metriques d'Engagement Client
print(f"\n📱 METRIQUES D'ENGAGEMENT CLIENT:")
print("-" * 40)
print(f"Moyen Page Vues per Session: {df['vues_page_par_session'].mean():.1f}")
print(f"Moyen Rebond Rate: {df['taux_rebond'].mean():.3f} ({df['taux_rebond'].mean()*100:.1f}%)")
print(f"Moyen Support Tickets per Client: {df['tickets_support'].mean():.2f}")

# Identificatgraphiqueion des clients a haute valeur
high_valeur_threshold = df['total_depense'].quantile(0.9)
high_valeur_clients = df[df['total_depense'] >= high_valeur_threshold]
print(f"\n💎 A HAUTE VALEUR CLIENTS (Top 10%):")
print("-" * 40)
print(f"Nonmbre: {len(high_valeur_clients)} clients")
print(f"Moyen Depenses: CHF{high_valeur_clients['total_depense'].mean():.2f}")
print(f"Moyen Satisfaction: {high_valeur_clients['score_satisfaction'].mean():.1f}/10")
print(f"Le plus Common Abonnement: {high_valeur_clients['type_abonnement'].mode().iloc[0]}")
print(f"Le plus Common Appareil: {high_valeur_clients['type_appareil'].mode().iloc[0]}")


In [ ]:
# 3. Insights Cles et Commercial Recommetations
print("\n" + "=" * 80)
print("💡 INSIGHTS CLES ET RECOMMANDATIONS COMMERCIALES")
print("=" * 80)

# Insights sur le comportement des clients
print("🎯 CLIENT BEHAVIOR INSIGHTS:")
print("-" * 40)

# Comportement de depenses
high_spenders = df[df['total_depense'] > df['total_depense'].quantile(0.75)]
low_spenders = df[df['total_depense'] < df['total_depense'].quantile(0.25)]

print(f"• Eleve Spenders (Top 25%) vs Faible Spenders (Botam 25%):")
print(f"  - Eleve Spenders: {len(high_spenders)} clients, Avg: CHF{high_spenders['total_depense'].mean():.2f}")
print(f"  - Faible Spenders: {len(low_spenders)} clients, Avg: CHF{low_spenders['total_depense'].mean():.2f}")
print(f"  - Depenses Ratio: {high_spenders['total_depense'].mean() / low_spenders['total_depense'].mean():.1f}x")

# Modeles d'engagement
print(f"\n• Engagement Modeles:")
print(f"  - Eleve Engagement (Long Sessions): {len(df[df['duree_moyenne_session'] > df['duree_moyenne_session'].quantile(0.75)])} clients")
print(f"  - Faible Engagement (Short Sessions): {len(df[df['duree_moyenne_session'] < df['duree_moyenne_session'].quantile(0.25)])} clients")
print(f"  - Eleve Rebond Rate (>50%): {len(df[df['taux_rebond'] > 0.5])} clients ({len(df[df['taux_rebond'] > 0.5])/len(df)*100:.1f}%)")

# Analyse de satisfaction
print(f"\n• Analyse de Satisfaction:")
satisfaits_clients = df[df['score_satisfaction'] >= 8]
dissatisfaits_clients = df[df['score_satisfaction'] <= 4]
print(f"  - Clients Satisfaits (8+): {len(satisfaits_clients)} ({len(satisfaits_clients)/len(df)*100:.1f}%)")
print(f"  - Clients Non Satisfaits (≤4): {len(dissatisfaits_clients)} ({len(dissatisfaits_clients)/len(df)*100:.1f}%)")

# Recommetations Commerciales
print(f"\n📋 RECOMMANDATIONS STRATEGIQUES:")
print("-" * 40)

# Optimisation des revenus
print("💰 OPTIMISATION DES REVENUS:")
print("1. Se concentrer sur les segments de clients a haute valeur:")
print(f"   • Cibler les clients avec {high_valeur_clients['type_abonnement'].mode().iloc[0]} abonnements")
print(f"   • Prioriser {high_valeur_clients['type_appareil'].mode().iloc[0]} users pour premium features")
print(f"   • Implementer des programmes VIP pour top {len(high_valeur_clients)} clients")

# Sttauxgie d'abonnement
meilleur_abonnement = df.groupby('type_abonnement')['total_depense'].mean().idxmax()
print(f"\n2. Sttauxgie d'Abonnement:")
print(f"   • Promouvoir {meilleur_abonnement} abonnement as it shows plus eleve average depenses")
print(f"   • Creer des incitations a la mise a niveau pour Basique → Premium clients")
print(f"   • Developper des fonctionnalites Entreprise based on high-valeur client needs")

# Expansion geographique
ville_top = df.groupby('ville')['total_depense'].mean().idxmax()
print(f"\n3. Expansion Geographique:")
print(f"   • Etendre les efpourts de marketing in {ville_top} (plus eleve avg depenses)")
print(f"   • Replicatgraphiquee successful sttauxgies from top-perpourming villes")
print(f"   • Considerer les partenariats locaux in high-valeur markets")

# Experience client
print(f"\n4. Ameliorations de l'Experience Client:")
print(f"   • Aborder high bounce taux ({len(df[df['taux_rebond'] > 0.5])/len(df)*100:.1f}% of clients)")
print(f"   • Ameliorer la duree de session pour etretter engagement")
print(f"   • Implementer des enquetes de satisfaction pour clients scoring ≤4")

# Optimisation de l'engagement
print(f"\n5. Optimisation de l'Engagement:")
print(f"   • Developper des fonctionnalites mobile-first (60% mobile users)")
print(f"   • Creer du contenu personnalise based on segments d'age")
print(f"   • Implementer des programmes de fidelite pour repeat clients")

print(f"\n📊 METRIQUES DE SUCCES A SUIVRE:")
print("-" * 40)
print("• Valeur Vie Client (CLV)")
print("• Mensuel Recurring Revenus (MRR)")
print("• Cout d'Acquisition Client (CAC)")
print("• Taux de Churn by segment")
print("• Net Promouvoirr Score (NPS)")
print("• Moyen Revenus Per Utilisateur (ARPU)")


In [ ]:
# 4. Final Resume Tableau de bord
print("\n" + "=" * 80)
print("📊 TABLEAU DE BORD DE SYNTHESE FINALE")
print("=" * 80)

# Creer un tableau de bord de synthese
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Revenus by Abonnement Type
revenue_by_sub = df.groupby('type_abonnement')['total_depense'].sum()
revenue_by_sub.graphique(kind='pie', ax=axes[0,0], auapct='%1.1f%%', startangle=90)
axes[0,0].set_title('Revenus Distribution by Abonnement Type', fontweight='bold')
axes[0,0].set_ylaetrel('')

# 2. Client Nonmbre by Ville
ville_countgraphiques = df['ville'].valeur_countgraphiques().head(5)
ville_countgraphiques.graphique(kind='bargraphique', ax=axes[0,1], color='lightblue')
axes[0,1].set_title('Top 5 Cities by Client Nonmbre', fontweight='bold')
axes[0,1].set_xlaetrel('Ville')
axes[0,1].set_ylaetrel('Nonmbre de Clients')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Score de Satisfaction Distribution
df['score_satisfaction'].hist(bins=10, ax=axes[0,2], color='lightgreen', alpha=0.7, edgecolor='black')
axes[0,2].set_title('Score de Satisfaction Distribution', fontweight='bold')
axes[0,2].set_xlaetrel('Score de Satisfaction')
axes[0,2].set_ylaetrel('Nonmbre de Clients')
axes[0,2].axvlinegraphique(df['score_satisfaction'].mean(), color='red', linegraphiquestyle='--', 
                  laetrel=f'Moyenne: {df["score_satisfaction"].mean():.1f}')
axes[0,2].legende()

# 4. Depenses by Age Segment
age_depenses = df.groupby('age_segment')['total_depense'].mean()
age_depenses.graphique(kind='bargraphique', ax=axes[1,0], color='gold')
axes[1,0].set_title('Moyen Depenses by Age Segment', fontweight='bold')
axes[1,0].set_xlaetrel('Age Segment')
axes[1,0].set_ylaetrel('Moyen Atal Depense (CHF)')
axes[1,0].tick_params(axis='x', rotation=45)

# 5. Appareil Type Distribution
device_countgraphiques = df['type_appareil'].valeur_countgraphiques()
device_countgraphiques.graphique(kind='bargraphique', ax=axes[1,1], color='lightcoral')
axes[1,1].set_title('Client Distribution by Appareil Type', fontweight='bold')
axes[1,1].set_xlaetrel('Appareil Type')
axes[1,1].set_ylaetrel('Nonmbre de Clients')
axes[1,1].tick_params(axis='x', rotation=45)

# 6. Cle Metriques Resume
axes[1,2].axis('off')
resume_text = f"""
KEY METRIQUES SUMMARY

👥 Atal Clients: {total_clients:,}
💰 Atal Revenus: CHF{total_revenue:,.0f}
💵 Avg Revenus/Client: CHF{avg_revenue_per_client:.0f}
😊 Avg Satisfaction: {avg_satisfaction:.1f}/10
⏱️ Avg Session Duree: {duree_moyenne_session:.1f} min

INSIGHTS PRINCIPAUX:
• {meilleur_abonnement} abonnement perpourme le mieux
• {ville_top} montre les depenses les plus elevees
• {len(high_valeur_clients)} clients a haute valeur identifies
• {len(satisfaits_clients)} clients satisfaits (8+ note)
"""

axes[1,2].text(0.1, 0.9, resume_text, transpourm=axes[1,2].transAxes, 
               fontsize=12, verticalalignment='top', fontfamily='monospace',
               bboxgraphique=dict(boxgraphiquestyle="round,pad=0.3", facecolor="lightgray", alpha=0.5))

plt.tight_layout()
plt.show()

print("✅ EXPLORATION DES DONNEES TERMINEE!")
print("🎉 Votre analyse complete du comportement des clients est prete!")
print("📈 Utiliser these insights a drive donnees-driven business decisions!")
